# Updates

In DataJoint, **updates are deliberately rare**.
This reflects a core principle of the [Relational Workflow Model](../20-concepts/05-workflows.md): computed results should remain consistent with their inputs.

## Why Updates Are Discouraged

Consider what happens when you update an upstream value:
- Downstream computed results were derived from the **old** value
- After the update, they coexist with the **new** value
- The relationship between inputs and outputs is broken—**computational validity** is violated

The proper approach for most corrections is the **delete-reinsert-repopulate** pattern:

1. **Delete** the incorrect data (cascading removes all dependent computations)
2. **Insert** the corrected data
3. **Repopulate** to regenerate downstream results with the new inputs

This ensures every computed result accurately reflects its inputs.

## When `update1` Is Appropriate

The `update1` method exists for cases where in-place correction is truly appropriate:

- **Non-scientific metadata**: Notes, comments, administrative fields that don't affect computations
- **Corrections without downstream impact**: Fields that no computed table depends on
- **Fixing typos**: In descriptive text fields

The key question: *Does any downstream computation depend on this value?*
If yes, use delete-reinsert. If no, `update1` may be appropriate.

## Syntax

```python
table.update1(record)
```

The `record` is a dictionary containing:
- All primary key values (to identify which row to update)
- The attribute(s) to update with their new values

The entry must already exist—`update1` will raise an error if it doesn't.

In [2]:
import datajoint as dj
schema = dj.schema('test_update')

@schema
class Student(dj.Manual):
    definition = """
    student_id : int
    ---
    full_name : varchar(100)   # last_name, first_name middle_name
    phone="": varchar(20)
    sex : enum('female', 'male')
    """

In [3]:
Student.insert1(dict(student_id=303, full_name="Rosen, Rose", sex="female"))
Student.insert1(dict(student_id=304, full_name="Rosen, Rose", sex="male", phone="(813)555-3744"))

In [4]:
Student()

student_id,"full_name last_name, first_name middle_name",phone,sex
303,"Rosen, Rose",,female
304,"Rosen, Rose",(813)555-3744,male


Update specific values by providing the primary key and the fields to change:

In [5]:
Student.update1(dict(student_id=303, phone="(813)555-7133"))
Student.update1(dict(student_id=304, full_name="Ramesh, Henry"))
Student()

student_id,"full_name last_name, first_name middle_name",phone,sex
303,"Rosen, Rose",(813)555-7133,female
304,"Ramesh, Henry",(813)555-3744,male


## Error Handling

`update1` enforces strict requirements. Attempting to update a non-existent entry raises an error:

In [6]:
Student.update1(dict(student_id=305, phone="(800)555-3377"))

DataJointError: Update entry must exist.

In [7]:
Student.update1(dict(phone="(800)555-3377"))

DataJointError: The argument of update1 must supply all primary key values.

Similarly, omitting the primary key raises an error because DataJoint cannot identify which row to update.

## Summary

| Scenario | Recommended Approach |
|----------|---------------------|
| Correcting data that affects computations | Delete → Insert → Repopulate |
| Fixing typos in descriptive fields | `update1` |
| Changing administrative metadata | `update1` |
| Any doubt about downstream impact | Delete → Insert → Repopulate |

The conservative approach—delete and reinsert—is almost always safer.
Use `update1` only when you are certain the change has no computational consequences.